# Import the necessary functions and classes.

In [ ]:
from db_connector import open_table

from data_transform import DataTransform

from data_frame_info import DataFrameInfo

from plotter import Plotter

from data_frame_transform import Transform

# Retrieve table.

In [ ]:
raw_table = open_table()

# Call methods and classes.

In [ ]:
transform_call = DataTransform(raw_table)

# Remove strings from the 'term' column.
transform_call.remove_term_column_strings()

# Convert all date columns to datetime format.
transform_call.iterate_through_columns()

# Initialise a DataFrameInfo call with the raw dataframe.
find_info = DataFrameInfo(raw_table)

# Create table of percentage null values in the raw dataframe.
null_percentages_table = find_info.percentage_null_values()
print(null_percentages_table)


# Sort data based on the data information.

In [ ]:
# Columns with null values:
all_null_columns = ['mths_since_last_record', 'mths_since_last_major_derog', 'next_payment_date', 'mths_since_last_delinq', 'employment_length', 'last_payment_date', 'last_credit_pull_date','term', 'int_rate', 'funded_amount', 'collections_12_mths_ex_med']

 # Columns with >50% null values:
highest_null_proportion_columns = ['mths_since_last_record', 'mths_since_last_major_derog', 'next_payment_date', 'mths_since_last_delinq']

# Categorical columns:
categorical_columns = ['employment_length', 'term'] # Contains null values

# Date columns
date_columns = ['last_payment_date', 'last_credit_pull_date'] # Contains null values
print(date_columns)

# Columns with <10% null values:
low_null_columns = ['int_rate', 'funded_amount', 'last_payment_date', 'last_credit_pull_date', 'collections_12_mths_ex_med']


# Assess skewness of columns with low percentage null values.

In [ ]:
#FIX
skew_table = []
find_info.column_skew(low_null_columns)


# Sort the low percentage null columns by skewness.

In [ ]:
# Columns with <1 skew:
low_skew_columns = ['int_rate', 'funded_amount']

# Columns with >1 skew:
high_skew_columns = ['collections_12_mths_ex_med']

# Initialise a dataframe transform call.

In [ ]:
data_frame_transform_call = DataFrameTransform(raw_table, find_info, highest_null_proportion_columns, low_skew_columns, categorical_columns, high_skew_columns, date_columns)

# Remove the null values in the dataframe.

In [ ]:
# Drop all columns with >50% null values.
data_frame_transform_call.drop_columns()

# Impute null values in categorical columns with the mode.
data_frame_transform_call.impute_with_mode()

# Impute null values in columns with >1 skew with the median.
data_frame_transform_call.impute_with_median()

# Drop rows with null values in columns with <1% null values.
data_frame_transform_call.drop_rows()

# Impute remaining null values in columns with <10% null values with the mean.
no_null_table = data_frame_transform_call.impute_with_mean()

# Visualise missing values before and after removal of null values.

In [ ]:
# Call the plotter class with the raw data frame and produce matrix.
visualise = Plotter(raw_table)
visualise.missing_data()

# Call the plotter class with the null-removed
visualise = Plotter(no_null_table)
visualise.missing_data()

# Assess dataframe after null removal.

In [ ]:
# Call DataFrameInfo class with latest dataframe.
new_info = DataFrameInfo(no_null_table)

# View data types within new dataframe.
new_table_data_types = new_info.find_column_types()

# Assess skew of new dataframe.
all_skew = new_info.column_skew()

# Visualise the skewness of the dataframe.
new_info.get_histogram()

# List all column names in the dataframe.
all_column_names = list(no_null_table)

# Call the DataFrameTransform class after removal of nulls in the dataframe.
transform_post_null_removal = DataFrameTransform(raw_table, null_percentages_table, highest_null_proportion_columns, low_skew_columns, categorical_columns, high_skew_columns, date_columns, all_skew)

# Call the DataFrameTransform class after removal of nulls in the dataframe.
transform_post_null_removal = DataFrameTransform(raw_table, null_percentages_table, highest_null_proportion_columns, low_skew_columns, categorical_columns, high_skew_columns, date_columns, all_skew)

transform_post_null_removal.boxcox_transform_skew(all_skew)

# Import null-removed table from local saved location:

In [136]:
import pandas as pd
# Load CSV file as dataframe from local machine
def no_null_table():
    with open('../../file_saves/null_removed_table.csv', 'r') as file:
        saved_loan_payments_table = pd.read_csv(file)
    return saved_loan_payments_table

clean_table = no_null_table()



,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,last_credit_pull_date,collections_12_mths_ex_med,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36.0,7.49,248.82,A,A4,...,512.21,0.000000,0.00,0.00,2022-01,248.82,2022-01,0.0,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36.0,6.99,407.52,A,A3,...,787.94,0.000000,0.00,0.00,2022-01,407.52,2022-01,0.0,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36.0,7.49,497.63,A,A4,...,824.54,0.000000,0.00,0.00,2021-10,12850.16,2021-10,0.0,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36.0,14.31,514.93,C,C4,...,947.47,0.000000,0.00,0.00,2021-06,13899.67,2021-06,0.0,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36.0,6.03,456.54,A,A1,...,770.93,0.000000,0.00,0.00,2022-01,456.54,2022-01,0.0,1,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54146,76597,76583,5000,5000.0,1775.0,36.0,9.01,159.03,B,B2,...,724.82,0.000000,0.00,0.00,2016-07,160.61,2016-07,0.0,1,INDIVIDUAL
54147,142608,74724,5000,5000.0,2350.0,36.0,10.59,162.73,C,C2,...,826.31,104.942381,0.00,0.00,2016-10,490.01,2016-09,0.0,1,INDIVIDUAL
54148,117045,70978,3500,3500.0,2225.0,36.0,7.43,108.77,A,A2,...,415.37,0.000000,0.00,0.00,2016-09,110.58,2013-05,0.0,1,INDIVIDUAL
54149,88854,70699,5000,5000.0,225.0,36.0,7.43,155.38,A,A2,...,174.18,0.000000,0.00,0.00,2014-03,0.00,2013-05,0.0,1,INDIVIDUAL
